In [5]:
import os
from bs4 import BeautifulSoup
from natasha import NamesExtractor  
import json
extractor = NamesExtractor()
import re
#регулярка для имён
fio = re.compile("[А-ЯЁ][а-яё]+ [А-ЯЁ]\.[А-ЯЁ]\.")
import pandas as pd

### Собираем готовые метаданные

Функция, которая возвращает эксплицитно указанные метаданные и очищенный от тегов текст, а также создаёт словарь, который включает другие метаданные (прокурора, адвоката, секретаря, обвиняемого):

In [6]:
def parse_xml(path):   # парсим xml
    meta_data = {'number': path.split("/")[-1], 'court': 'not_found', 'judge': 'not_found', 'prosecutor': 'not_found', 'advocate': 'not_found', 'secretary': 'not_found', 'accused': 'not_found', 'result': 'not_found', 'category': 'not_found'}
    with open(path, 'r', encoding='utf-8')as f: 
                
        soup = BeautifulSoup(f.read(), 'html.parser')
        meta_data['court'] = soup.court.string     # добавляем в метаданные то, что имеется в разметке
        meta_data['judge'] = soup.judge.string
        meta_data['result'] = soup.result.string
        meta_data['category'] = soup.category.string
        
        html = []
        for line in soup.body:
            line = line.replace('[', '')
            line = line.replace(']', '') 
            html.append(line)
        true_html = ' '.join(html)
                
        html_soup = BeautifulSoup(true_html, 'html.parser') 
    return meta_data, html_soup.get_text()

Функция на всякий случай -- если потребуется достать что-то с разметкой:

In [7]:
def open_with_xml(path):
    text = ""
    with open(path, 'r', encoding='utf-8')as f: 
        for line in f.readlines():
            text += line
    return text

Функция, которая разбивает текст на части (вступление, основная часть, заключение):

In [38]:
def get_parts(text):    # делим на части
    lines = [line for line in text.split('\n')]
    beg, end = 0, 0 
    for num, line in enumerate(lines):
        if 'установил' in line.lower() or 'у с т а н о в и л' in line.lower(): 
            beg = num + 1
        if 'приговорил' in line.lower() or 'п р и г о в о р и л' in line.lower() or 'определил' in line.lower() or 'о п р е д е л и л' in line.lower()  or 'решил' in line.lower() or 'р е ш и л' in line.lower() or "постановил" in line.lower() or "п о с т а н о в и л" in line.lower(): 
            end = num + 1 
        if beg and end:
            beginning = [stroka.strip() for stroka in lines[:beg]]
            main_part = [stroka.strip() for stroka in lines[beg:end]]
            ending = [stroka.strip() for stroka in lines[end:]]
            return [' '.join(main_part), '\n'.join(beginning), '\n'.join(ending)]
        
        # для извлечения метаданных берём get_parts(text)[1:]
        # для всего остального - get_parts(text)[0]

Функция, которая достаёт из строчки русское имя в формате И.И. Иванов:

In [9]:
def get_names(line):
    name = ""
    matches = extractor(line)
    for match in matches:
        fact = match.fact.as_json 
        name = fact["first"]+"."+fact["middle"]+". "+fact["last"].capitalize()
    return name

Функция, которая:
- разбивает текст на три основные части с помощью get_parts
- разбивает вступление либо по запятой (по умолчанию), либо по переносу строки (если запятая не отделяет одни сущности от других)

In [10]:
def splitting_text(path):
    trouble_counter = 0
    text = get_parts(parse_xml(path)[1])[1]
    lines = [line for line in text.split(",") if line.strip()]
    for line in lines:
        if len(fio.findall(line)) > 1:
            trouble_counter += 1
    if trouble_counter != 0:
        lines = [line for line in text.split("\n") if line.strip()]
    return lines       

Функция, которая достаёт строки рядом с нужными нам ключевыми словами, в которых содержатся ФИО:

In [11]:
def meta_lines(path):
    lines = splitting_text(path) 
    add_meta = []
    for i, line in enumerate(lines):  
        if "защитник" in line.lower() or "адвокат" in line.lower(): 
            if fio.findall(line):
                add_meta.append(("advocate", line)) 
            else:
                add_meta.append(("advocate", line+lines[i+1])) 
        if "прокурор" in line.lower() or "обвинител" in line.lower():  
            if fio.findall(line):
                add_meta.append(("prosecutor", line)) 
            else:
                add_meta.append(("prosecutor", line+lines[i+1]))  
        if "секретар" in line.lower():
            if fio.findall(line):
                add_meta.append(("secretary", line)) 
            else:
                add_meta.append(("secretary", line+lines[i+1])) 
        if "подсудим" in line.lower() or "в отношении" in line.lower():
            if fio.findall(line):
                add_meta.append(("accused", line)) 
            else:
                if i+1 < len(lines):
                    add_meta.append(("accused", line+lines[i+1])) 
    return add_meta

Функция, которая достаёт из полученных выше строк ФИО формата: ("роль в суде", "ФИО"):

In [59]:
def get_meta_names(meta_lines):
    meta_names = []
    for line in meta_lines: 
        matches = extractor(line[1]) 
        if matches:
            try:
                for match in matches:
                    fact = match.fact.as_json 
                    meta_names.append((line[0], (fact["last"].capitalize()+" "+fact["first"]+"."+fact["middle"]+".")))
            except KeyError:
                if fio.findall(line[1]): 
                    meta_names.append((line[0], fio.findall(line[1])[0]))
        elif fio.findall(line[1]): 
            meta_names.append((line[0], fio.findall(line[1])[0]))
    return meta_names

Функция, которая:
- приводит список с ролями и именами к упорядоченному списку без повторений в нём
- достаёт готовые метаданные с помощью parse_xml в формате словаря
- объединяет те записи из списка, для которых первое значение (роль) совпадают: (адвокат, "Петров А.П.") и (адвокат, "Семёнова Д.В.") приводятся к форме (адвокат, "Петров А.П., Семенова Д.В.")
- добавляет полученные записи из списка в словарь метаданных

In [13]:
def names_to_meta(path):
    meta_data = parse_xml(path)[0]
    updated_names = []
    unique_names = list(set(get_meta_names(meta_lines(path))))
    unique_names.sort()
    #объединение кортежей с одинаковыми нулевыми значениями 
    for i, name in enumerate(unique_names):
        if i+1 < len(unique_names):
            if name[0] == unique_names[i+1][0]:
                updated_names.append((name[0], name[1]+", "+unique_names[i+1][1]))
            elif name[0] != unique_names[i-1][0] or i == 0: 
                updated_names.append((name[0], name[1]))
        elif i+1 >= len(unique_names) and name[0] != unique_names[i-1][0]:
            updated_names.append((name[0], name[1]))
        elif len(unique_names) == 1:
            updated_names.append((name[0], name[1]))

    for name in updated_names:
        meta_data[name[0]] = name[1] 
    return meta_data

Функция, которая достаёт ссылки, которые лежат внутри папки (для папки без вложенных папок):

In [14]:
def links_for_folder(path_to_folder):
    paths = []
    for f in os.walk(path_to_folder):
        for path in f[2]:
            paths.append(path_to_folder+path)
    return paths

Функция, которая возвращает словари метаданных для всех документов в папке и также возвращает количество ошибок во время извлечения:

In [28]:
def dicts_for_folder(path_to_folder):
    kerror_counter = 0
    kerror_cases = []
    terror_counter = 0
    terror_cases = []
    folder_dicts = []
    for path in links_for_folder(path_to_folder): 
        try:
            folder_dicts.append(names_to_meta(path))
        except KeyError: 
            kerror_counter += 1
            kerror_cases.append(path)
            continue
        except TypeError: 
            terror_counter += 1
            terror_cases.append(path)
            continue
    return folder_dicts, (kerror_counter, kerror_cases), (terror_counter, terror_cases)

Функция, которая создаёт для заданной папки датафрейм с метаинформацией:

In [16]:
def df_for_folder(path_to_folder):
    case_list = dicts_for_folder(path_to_folder)[0]
    base_df = pd.DataFrame.from_dict(case_list[0], orient = "index")
    folder_df = base_df.T
    for case in case_list[1:]:
        folder_df.loc[len(folder_df)] = pd.DataFrame.from_dict(case, orient = "index").T.loc[0]   
    return folder_df

In [266]:
dicts_for_folder("/home/laidhimonthegreen/Документы/coursework/regexp/belinskij/")

([{'number': '105539785.xml',
   'court': 'Белинский районный суд (Пензенская область)',
   'judge': 'Саунин Николай Владимирович',
   'prosecutor': 'Касаткина Н.В.',
   'advocate': 'Карцева Л.В.',
   'secretary': 'Любимкиной Т.В.',
   'accused': 'Матвеев С.И.',
   'result': 'Обвинительный приговор',
   'category': '222 ч.1'},
  {'number': '106623286.xml',
   'court': 'Белинский районный суд (Пензенская область)',
   'judge': 'Круглякова Букина Людмила Васильевна',
   'prosecutor': 'Касаткина Н.В.',
   'advocate': 'Кердяшов С.М.',
   'secretary': 'Горшковой М.А.',
   'accused': 'Грязнов А.В., Грязнова А.В.',
   'result': 'Обвинительный приговор',
   'category': '158 ч.2 п.б'},
  {'number': '104670045.xml',
   'court': 'Белинский районный суд (Пензенская область)',
   'judge': 'Круглякова Букина Людмила Васильевна',
   'prosecutor': 'Касаткина Н.В.',
   'advocate': 'Кердяшов С.М.',
   'secretary': 'Шебурова В.И.',
   'accused': 'Артемов А.И., Артемова А.И.',
   'result': 'Обвинительный 

In [267]:
df_for_folder("/home/laidhimonthegreen/Документы/coursework/regexp/belinskij/")

,number,court,judge,prosecutor,advocate,secretary,accused,result,category
0,105539785.xml,Белинский районный суд (Пензенская область),Саунин Николай Владимирович,Касаткина Н.В.,Карцева Л.В.,Любимкиной Т.В.,Матвеев С.И.,Обвинительный приговор,222 ч.1
1,106623286.xml,Белинский районный суд (Пензенская область),Круглякова Букина Людмила Васильевна,Касаткина Н.В.,Кердяшов С.М.,Горшковой М.А.,"Грязнов А.В., Грязнова А.В.",Обвинительный приговор,158 ч.2 п.б
2,104670045.xml,Белинский районный суд (Пензенская область),Круглякова Букина Людмила Васильевна,Касаткина Н.В.,Кердяшов С.М.,Шебурова В.И.,"Артемов А.И., Артемова А.И.",Обвинительный приговор,264 ч.2
3,104552258.xml,Белинский районный суд (Пензенская область),Саунин Николай Владимирович,Чуглина М.А.,"Кердяшов С.М., Ромакина Н.А.",Любимкиной Т.В.,"Захаров В.В., Сбруякина В.Н.",Обвинительный приговор,158 ч.2
4,106165849.xml,Белинский районный суд (Пензенская область),Чуглина Елена Александровна,Кудряшов Ю.В.,Карцева Л.В.,Горшковой М.А.,Холматова М.М.,Обвинительный приговор,291 ч.1
5,400056467.xml,Белинский районный суд (Пензенская область),Саунин Николай Владимирович,Тюнина Ж.В.,Кердяшов С.М.,Любимкиной Т.В.,Ефимкин В.П.,Вынесен ПРИГОВОР,264 ч.1
6,104763033.xml,Белинский районный суд (Пензенская область),Чуглина Елена Александровна,Кудряшов Ю.В.,Ердяшова С.М.,Горшковой М.А.,Кусакина А.И.,Обвинительный приговор,158 ч.2 п.б
7,100788062.xml,Белинский районный суд (Пензенская область),Саунин Николай Владимирович,Касаткина Н.В.,"Кердяшов С.М., Ромакина Н.А.",Ромакина В.А.,"Ильин М.А., Ильин С.И.",Обвинительный приговор,158 ч.2; 158 ч.2 п.б
8,104333229.xml,Белинский районный суд (Пензенская область),Кисткин Владимир Анатольевич,Касаткина Н.В.,Кердяшов С.М.,Ромакина В.А.,Ульченко В.Г.,Обвинительный приговор,223 ч.1; 119 ч.1; 222 ч.1
9,106535123.xml,Белинский районный суд (Пензенская область),Саунин Николай Владимирович,Касаткина Н.В.,Карцева Л.В.,Любимкиной Т.В.,Лазарев А.В.,Обвинительный приговор,111 ч.1


### Результаты:
Из 57 дел фатальные ошибки были в двух (не вполне стандартно оформленных, думаю, что их можно будет в общей картине просто пропустить).

При анализе 20 верхних дел вручную попадались небольшие ошибки программы, сейчас они исправлены.
Надо проверить на других судах, если там будет примерно такая же точность, будет ок. 

## Тестируем на других судах

In [29]:
%%time

dicts_for_folder("/home/laidhimonthegreen/Документы/coursework/regexp/primorskij/2010/")

CPU times: user 49.5 s, sys: 59.9 ms, total: 49.5 s
Wall time: 49.5 s


([{'number': '100209888.xml',
   'court': 'Приморский районный суд (Архангельская область)',
   'judge': 'Ибрагимова Надежда Алексеевна',
   'prosecutor': 'Худяков Е.А.',
   'advocate': 'Коломийцев А.А.',
   'secretary': 'Селякова С.Ю.',
   'accused': 'Селезнев С.В., Тесленко А.В.',
   'result': 'Обвинительный приговор',
   'category': '158 ч.2 п.а'},
  {'number': '103411037.xml',
   'court': 'Приморский районный суд (Архангельская область)',
   'judge': 'Романова Наталья Владимировна',
   'prosecutor': 'not_found',
   'advocate': 'not_found',
   'secretary': 'Цыганова И.А.',
   'accused': 'not_found',
   'result': 'Иск (заявление) удовлетворен (в том числе частично)',
   'category': 'Трудовые споры (независимо от форм собственности работодателя): - об оплате труда'},
  {'number': '100208908.xml',
   'court': 'Приморский районный суд (Архангельская область)',
   'judge': 'Климова Анна Анатольевна',
   'prosecutor': 'Кольцова А.В., Стафеева А.А.',
   'advocate': 'not_found',
   'secreta

12 и 69 ошибок. Первое число допустимо (меньше 5 процентов), над вторым поработаем. Добавим слово "определил" для парсинга.

Пустим заново:

In [37]:
%%time

dicts_for_folder("/home/laidhimonthegreen/Документы/coursework/regexp/primorskij/2010/")

CPU times: user 1min 1s, sys: 75.9 ms, total: 1min 1s
Wall time: 1min 1s


([{'number': '100209888.xml',
   'court': 'Приморский районный суд (Архангельская область)',
   'judge': 'Ибрагимова Надежда Алексеевна',
   'prosecutor': 'Худяков Е.А.',
   'advocate': 'Коломийцев А.А.',
   'secretary': 'Селякова С.Ю.',
   'accused': 'Селезнев С.В., Тесленко А.В.',
   'result': 'Обвинительный приговор',
   'category': '158 ч.2 п.а'},
  {'number': '103411037.xml',
   'court': 'Приморский районный суд (Архангельская область)',
   'judge': 'Романова Наталья Владимировна',
   'prosecutor': 'not_found',
   'advocate': 'not_found',
   'secretary': 'Цыганова И.А.',
   'accused': 'not_found',
   'result': 'Иск (заявление) удовлетворен (в том числе частично)',
   'category': 'Трудовые споры (независимо от форм собственности работодателя): - об оплате труда'},
  {'number': '100208908.xml',
   'court': 'Приморский районный суд (Архангельская область)',
   'judge': 'Климова Анна Анатольевна',
   'prosecutor': 'Кольцова А.В., Стафеева А.А.',
   'advocate': 'not_found',
   'secreta

15 и 24. Посмотрим на ошибки. Да, надо добавить "о п р е д е л и л".
Ещё раз:

In [39]:
%%time

dicts_for_folder("/home/laidhimonthegreen/Документы/coursework/regexp/primorskij/2010/")

CPU times: user 54.6 s, sys: 76 ms, total: 54.7 s
Wall time: 54.7 s


([{'number': '100209888.xml',
   'court': 'Приморский районный суд (Архангельская область)',
   'judge': 'Ибрагимова Надежда Алексеевна',
   'prosecutor': 'Худяков Е.А.',
   'advocate': 'Коломийцев А.А.',
   'secretary': 'Селякова С.Ю.',
   'accused': 'Селезнев С.В., Тесленко А.В.',
   'result': 'Обвинительный приговор',
   'category': '158 ч.2 п.а'},
  {'number': '103411037.xml',
   'court': 'Приморский районный суд (Архангельская область)',
   'judge': 'Романова Наталья Владимировна',
   'prosecutor': 'not_found',
   'advocate': 'not_found',
   'secretary': 'Цыганова И.А.',
   'accused': 'not_found',
   'result': 'Иск (заявление) удовлетворен (в том числе частично)',
   'category': 'Трудовые споры (независимо от форм собственности работодателя): - об оплате труда'},
  {'number': '100208908.xml',
   'court': 'Приморский районный суд (Архангельская область)',
   'judge': 'Климова Анна Анатольевна',
   'prosecutor': 'Кольцова А.В., Стафеева А.А.',
   'advocate': 'not_found',
   'secreta

Хорошо. В оставшихся 16 документах в подавляющем большинстве просто "пустые дела" без основной части. Количество key error всё ещё приемлемо.

In [41]:
df_for_folder("/home/laidhimonthegreen/Документы/coursework/regexp/primorskij/2010/")

,number,court,judge,prosecutor,advocate,secretary,accused,result,category
0,100209888.xml,Приморский районный суд (Архангельская область),Ибрагимова Надежда Алексеевна,Худяков Е.А.,Коломийцев А.А.,Селякова С.Ю.,"Селезнев С.В., Тесленко А.В.",Обвинительный приговор,158 ч.2 п.а
1,103411037.xml,Приморский районный суд (Архангельская область),Романова Наталья Владимировна,not_found,not_found,Цыганова И.А.,not_found,Иск (заявление) удовлетворен (в том числе част...,Трудовые споры (независимо от форм собственнос...
2,100208908.xml,Приморский районный суд (Архангельская область),Климова Анна Анатольевна,"Кольцова А.В., Стафеева А.А.",not_found,not_found,Стафеева А.А.,Отменено с направлением по подведомственности,19.16
3,100209871.xml,Приморский районный суд (Архангельская область),Климова Анна Анатольевна,Кольцова А.В.,Сокольцов Э.В.,Бобровской А.П.,"Киркина С.В., Сокольцов Э.В.",Прекращено по другим основаниям (по прим. к ст...,264 ч.1
4,100208903.xml,Приморский районный суд (Архангельская область),Ибрагимова Надежда Алексеевна,"Климова С.Л., Лобанов А.Н.",not_found,not_found,"Климов С.Л., Климова С.Л.",Отменено возвращением на новое рассмотрение,20.11 ч.1
5,100209861.xml,Приморский районный суд (Архангельская область),Тарнаев Павел Владимирович,Меркулов П.В.,Минин Ю.Г.,Пятиной Т.И.,Родичева А.П.,Обвинительный приговор,238 ч.1
6,100208920.xml,Приморский районный суд (Архангельская область),Шитикова Ольга Александровна,not_found,Лебединский А.Л.,Селякова С.Ю.,Беляйцев И.Е.,Оставлено без изменения,12.27 ч.2
7,100209606.xml,Приморский районный суд (Архангельская область),Стрекаловская Наталья Александровна,not_found,not_found,"Ерофеев К.А., Ерофеева В.А.",not_found,Иск (заявление) удовлетворен (в том числе част...,"Споры, связанные с наследованием имущества"
8,100209586.xml,Приморский районный суд (Архангельская область),Познянский Станислав Анатольевич,not_found,"Минин А.А., Ушакова Е.Ф.","Минин А.А., Ушакова Е.Ф.",not_found,Иск (заявление) удовлетворен (в том числе част...,Споры о праве собственности на землю
9,100209850.xml,Приморский районный суд (Архангельская область),Ибрагимова Надежда Алексеевна,Зыкин Д.Н.,Минин Ю.Г.,Селякова С.Ю.,Панкратова А.О.,Обвинительный приговор,161 ч.2 п.г


Много дел без, например, прокурора. Это ошибка или так и должно быть?

In [43]:
get_parts(parse_xml("/home/laidhimonthegreen/Документы/coursework/regexp/primorskij/2010/100208920.xml")[1])[1] 

'\n\nР Е Ш Е Н И Е\nпо результатам рассмотрения жалобы на постановление\nпо делу об административном правонарушении\n\n/адрес/ /дата/ г.\n\nСудья Приморского районного суда Архангельской области Ибрагимова Н.А.\nпри секретаре Селяковой С.Ю.\nв присутствие лица, в отношении которого ведется производство по делу об административном правонарушении Беляйцева И.Е.\nзащитника адвоката Лебединского А.Л.\nпотерпевшей Т\nрассмотрев в судебном заседании дело об административном правонарушении в отношении\nБеляйцев И.Е., родившегося /дата/ г. в /адрес/,  , работающего  , зарегистрированного по адресу: /адрес/, фактически проживающего в /адрес/,\nпо жалобе защитника адвоката Лебединского А.Л. на постановление от /дата/ г. мирового судьи судебного участка № 1, исполняющего обязанности мирового судьи судебного участка № 2 Приморского района Архангельской области,\n\nУ С Т А Н О В И Л :'

Да, в административных делах он часто не упоминается. Проверим наш скрипт на ещё паре судов и будем считать это нашим минимально жизнеспособным продуктом. Надо будет всё ещё потестить его на точность, потому что пока что мы фиксили все ошибки, которые находили вручную (на 30-40 делах), но настоящей проверки не проводили. 

In [44]:
dicts_for_folder("/home/laidhimonthegreen/Документы/coursework/regexp/bauntovskij/2012/")

([{'number': '411325553.xml',
   'court': 'Баунтовский районный суд (Республика Бурятия)',
   'judge': 'Мохорова Сержуни Мироновна',
   'prosecutor': 'not_found',
   'advocate': 'not_found',
   'secretary': 'Батуева О.В.',
   'accused': 'not_found',
   'result': 'Отменено с прекращением производства',
   'category': '19.5 ч.12'},
  {'number': '106509919.xml',
   'court': 'Баунтовский районный суд (Республика Бурятия)',
   'judge': 'Мохорова Сержуни Мироновна',
   'prosecutor': 'not_found',
   'advocate': 'not_found',
   'secretary': 'Батуева О.В.',
   'accused': 'not_found',
   'result': 'Иск (заявление) удовлетворен (в том числе частично)',
   'category': 'Прочие исковые дела'},
  {'number': '106871089.xml',
   'court': 'Баунтовский районный суд (Республика Бурятия)',
   'judge': 'Мохорова Сержуни Мироновна',
   'prosecutor': 'Аригуновая Б.В., Батуева О.В.',
   'advocate': 'not_found',
   'secretary': 'Батуева О.В.',
   'accused': 'not_found',
   'result': 'Дело прекращено',
   'categ

По 7 и 2 ошибки соответственно. Нормально.

Проверим на большой папке (1500 документов). 

In [46]:
%%time

dicts_for_folder("/home/laidhimonthegreen/Документы/coursework/regexp/rossoshanskij/2012/")

CPU times: user 1min 47s, sys: 168 ms, total: 1min 48s
Wall time: 1min 48s


([{'number': '403979959.xml',
   'court': 'Россошанский районный суд (Воронежская область)',
   'judge': 'Крюков Сергей Александрович',
   'prosecutor': 'Лесних Д.В.',
   'advocate': 'not_found',
   'secretary': 'Лысенко Т.П.',
   'accused': 'Бахтин В.Н.',
   'result': 'Иск (заявление, жалоба) УДОВЛЕТВОРЕН ЧАСТИЧНО',
   'category': 'Прочие из публично-правовых отношений'},
  {'number': '104800041.xml',
   'court': 'Россошанский районный суд (Воронежская область)',
   'judge': 'Авраменко Александр Митрофанович',
   'prosecutor': 'Белоконев О.В.',
   'advocate': 'Черников И.Н.',
   'secretary': 'Ростопша Г.И.',
   'accused': 'Денисов М.В.',
   'result': 'Прекращено по другим основаниям (по прим. к ст. УК РФ)',
   'category': '166 ч.1'},
  {'number': '101815911.xml',
   'court': 'Россошанский районный суд (Воронежская область)',
   'judge': 'Крюков Сергей Александрович',
   'prosecutor': 'not_found',
   'advocate': 'not_found',
   'secretary': 'Лысенко Т.П.',
   'accused': 'not_found',
  

7 процентов ошибок, с этим можно работать. Если получится улучшить -- будет хорошо.
Посмотрим на ошибки. 

In [48]:
get_parts(parse_xml("/home/laidhimonthegreen/Документы/coursework/regexp/rossoshanskij/2012/101816894.xml")[1])[1]

'\nДело № 1-26/12\nПОСТАНОВЛЕНИЕ\nо прекращении уголовного дела\nгород Россошь \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa007 февраля 2012 года\nРоссошанский районный суд Воронежской области в составе председательствующего судьи Черника С.А., с участием:\nпрокурора - старшего помощника Россошанского межрайпрокурора /Репринцев К.И./,\nобвиняемого /Данилов А.С./,\nего защитника - адвоката /Никонова К.В./, представившей удостоверение № и ордер № от 07.02.2012 года,\nпри секретаре Сырянной К.Ю.,\nрассмотрев в стадии предварительного слушания в закрытом судебном заседании в помещении Россошанского районного суда материалы уголовного дела в отношении /Данилов А.С./,<Дат

In [49]:
meta_lines("/home/laidhimonthegreen/Документы/coursework/regexp/rossoshanskij/2012/101816894.xml")

[('prosecutor',
  ' с участием:\nпрокурора - старшего помощника Россошанского межрайпрокурора /Репринцев К.И./'),
 ('advocate', '\nего защитника - адвоката /Никонова К.В./'),
 ('secretary', '\nпри секретаре Сырянной К.Ю.'),
 ('accused',
  '\nрассмотрев в стадии предварительного слушания в закрытом судебном заседании в помещении Россошанского районного суда материалы уголовного дела в отношении /Данилов А.С./'),
 ('accused',
  ' на основании ч. 5 ст. 74 УК РФ условное осуждение по приговору Эртильского районного суда Воронежской области от 16.12.2004 года в отношении /Данилов А.С./ отменено'),
 ('accused',
  ' на основании ч. 5 ст. 74 УК РФ условное осуждение по приговору Эртильского районного суда Воронежской области от 11.11.2009 года в отношении /Данилов А.С./ отменено')]

In [58]:
names_to_meta("/home/laidhimonthegreen/Документы/coursework/regexp/rossoshanskij/2012/101816894.xml")

Matches(' с участием:\nпрокурора - старшего помощника Россошанского межрайпрокурора /Репринцев К.И./', [])
Matches('\nего защитника - адвоката /Никонова К.В./', [Match([MorphTagToken('Никонова', [27, 35), 'RU', 'I', [Form('никонов', Grams(NOUN,Sgtm,Surn,anim,gent,masc,sing)), Form('никонов', Grams(NOUN,Sgtm,Surn,accs,anim,masc,sing)), Form('никонов', Grams(NOUN,Sgtm,Surn,anim,femn,nomn,sing))]), MorphTagToken('К', [36, 37), 'RU', 'O', [Form('кандидат', Grams(Abbr,Fixd,NOUN,inan,masc,nomn,sing)), Form('кандидат', Grams(Abbr,Fixd,NOUN,gent,inan,masc,sing)), Form('кандидат', Grams(Abbr,Fixd,NOUN,datv,inan,masc,sing)), Form('кандидат', Grams(Abbr,Fixd,NOUN,accs,inan,masc,sing)), Form('кандидат', Grams(Abbr,Fixd,NOUN,ablt,inan,masc,sing)), Form('кандидат', Grams(Abbr,Fixd,NOUN,inan,loct,masc,sing)), Form('к', Grams(Abbr,Fixd,Init,NOUN,Name,Sgtm,anim,masc,nomn,sing)), Form('к', Grams(Abbr,Fixd,Init,NOUN,Name,Sgtm,anim,gent,masc,sing)), Form('к', Grams(Abbr,Fixd,Init,NOUN,Name,Sgtm,anim,datv,

KeyError: 'last'

Ок, проблемы с распознаванием ФИО. Добавим ещё один небольшой костыль и проверим, как будет работать теперь. 

In [60]:
names_to_meta("/home/laidhimonthegreen/Документы/coursework/regexp/rossoshanskij/2012/101816894.xml")

{'number': '101816894.xml',
 'court': 'Россошанский районный суд (Воронежская область)',
 'judge': 'Черник Сергей Алексеевич',
 'prosecutor': 'Репринцев К.И.',
 'advocate': 'Никонов К.В.',
 'secretary': 'Сырянной К.Ю.',
 'accused': 'Данилов А.С.',
 'result': 'Прекращено по другим основаниям (по прим. к ст. УК РФ)',
 'category': '228 ч.1'}

Отлично. Прогоним заново для вышеупомянутых судов. 

In [63]:
%%time

dicts_for_folder("/home/laidhimonthegreen/Документы/coursework/regexp/rossoshanskij/2012/")[1:]

CPU times: user 1min 50s, sys: 168 ms, total: 1min 50s
Wall time: 1min 50s


((0, []),
 (9,
  ['/home/laidhimonthegreen/Документы/coursework/regexp/rossoshanskij/2012/429051153.xml',
   '/home/laidhimonthegreen/Документы/coursework/regexp/rossoshanskij/2012/106853496.xml',
   '/home/laidhimonthegreen/Документы/coursework/regexp/rossoshanskij/2012/102501651.xml',
   '/home/laidhimonthegreen/Документы/coursework/regexp/rossoshanskij/2012/106855023.xml',
   '/home/laidhimonthegreen/Документы/coursework/regexp/rossoshanskij/2012/106853501.xml',
   '/home/laidhimonthegreen/Документы/coursework/regexp/rossoshanskij/2012/104390680.xml',
   '/home/laidhimonthegreen/Документы/coursework/regexp/rossoshanskij/2012/104390724.xml',
   '/home/laidhimonthegreen/Документы/coursework/regexp/rossoshanskij/2012/102501567.xml',
   '/home/laidhimonthegreen/Документы/coursework/regexp/rossoshanskij/2012/104799949.xml']))

Отлично выглядит! Посмотрим в базе данных, нет ли там какой-то совсем уж видной глазу дичи. 

In [64]:
df_for_folder("/home/laidhimonthegreen/Документы/coursework/regexp/rossoshanskij/2012/")

,number,court,judge,prosecutor,advocate,secretary,accused,result,category
0,403979959.xml,Россошанский районный суд (Воронежская область),Крюков Сергей Александрович,Лесних Д.В.,not_found,Лысенко Т.П.,Бахтин В.Н.,"Иск (заявление, жалоба) УДОВЛЕТВОРЕН ЧАСТИЧНО",Прочие из публично-правовых отношений
1,104800041.xml,Россошанский районный суд (Воронежская область),Авраменко Александр Митрофанович,Белоконев О.В.,Черников И.Н.,Ростопша Г.И.,Денисов М.В.,Прекращено по другим основаниям (по прим. к ст...,166 ч.1
2,101815911.xml,Россошанский районный суд (Воронежская область),Крюков Сергей Александрович,not_found,not_found,Лысенко Т.П.,not_found,Иск (заявление) удовлетворен (в том числе част...,"Споры, связанные с наследованием имущества"
3,106882086.xml,Россошанский районный суд (Воронежская область),Ворона Александр Владимирович,Галушка Ю.А.,not_found,Лаптиевой Л.И.,not_found,Иск (заявление) удовлетворен (в том числе част...,Прочие исковые дела
4,404422894.xml,Россошанский районный суд (Воронежская область),Гладько Борис Николаевич,not_found,Юдин А.А.,Литвинова Т.А.,not_found,"ОТКАЗАНО в удовлетворении иска (заявлении, жал...",Жалобы на неправ. дейст. (безд.) - должностных...
5,106880627.xml,Россошанский районный суд (Воронежская область),Гладько Борис Николаевич,not_found,not_found,Литвинова Т.А.,not_found,Иск (заявление) удовлетворен (в том числе част...,Иски о взыскании сумм по договору займа
6,104799961.xml,Россошанский районный суд (Воронежская область),Ворона Александр Владимирович,not_found,not_found,"Анохин Н.В., Жиляков Н.В.",not_found,Иск (заявление) удовлетворен (в том числе част...,Связанные с приватизацией жилой площади
7,101815968.xml,Россошанский районный суд (Воронежская область),Гладько Борис Николаевич,not_found,not_found,Литвинова Т.А.,not_found,Иск (заявление) удовлетворен (в том числе част...,"Другие споры, связанные с землепользованием"
8,106753762.xml,Россошанский районный суд (Воронежская область),Плакида Ирина Дмитриевна,not_found,not_found,Шаиповой Л.Н.,not_found,Иск (заявление) удовлетворен (в том числе част...,Иски о взыскании платы за жилую площадь и комм...
9,101815884.xml,Россошанский районный суд (Воронежская область),Гладько Борис Николаевич,Галушка Ю.А.,not_found,Литвинова Т.А.,Ветров А.С.,Иск (заявление) удовлетворен (в том числе част...,Прочие из публично-правовых отношений


Попадается много not found, но мы уже видели, что это нормально для административных дел. 
Прогоним на папке, где было 16 и 16 ошибок -- удалось ли улучшить работу?

In [65]:
dicts_for_folder("/home/laidhimonthegreen/Документы/coursework/regexp/primorskij/2010/")[1:]

((0, []),
 (16,
  ['/home/laidhimonthegreen/Документы/coursework/regexp/primorskij/2010/100209004.xml',
   '/home/laidhimonthegreen/Документы/coursework/regexp/primorskij/2010/100208995.xml',
   '/home/laidhimonthegreen/Документы/coursework/regexp/primorskij/2010/100209017.xml',
   '/home/laidhimonthegreen/Документы/coursework/regexp/primorskij/2010/100209534.xml',
   '/home/laidhimonthegreen/Документы/coursework/regexp/primorskij/2010/100208998.xml',
   '/home/laidhimonthegreen/Документы/coursework/regexp/primorskij/2010/100209603.xml',
   '/home/laidhimonthegreen/Документы/coursework/regexp/primorskij/2010/100208997.xml',
   '/home/laidhimonthegreen/Документы/coursework/regexp/primorskij/2010/100209003.xml',
   '/home/laidhimonthegreen/Документы/coursework/regexp/primorskij/2010/100209016.xml',
   '/home/laidhimonthegreen/Документы/coursework/regexp/primorskij/2010/100209007.xml',
   '/home/laidhimonthegreen/Документы/coursework/regexp/primorskij/2010/100208993.xml',
   '/home/laidhi

16 "пустых" дел не улучшились, но от ошибок удалось избавиться. 